encoding utf-8 

@File       : .py
@Contact    : username.you@email.address.com
@License    : (C)Copyright AIQ Solutions 2022

@ModifyTime     @Author     @Version        @Description
-----------     -------     --------        ------------
  username    1.0             None

Module Documentation: 

In [1]:
import pydicom
from dcmrtstruct2nii import dcmrtstruct2nii, list_rt_structs
import dicom2nifti
import nibabel as nib
import nrrd
import glob
import os
import numpy as np
from collections import OrderedDict
import shutil


In [2]:
# import warnings
# warnings.filterwarnings("ignore")

In [2]:
root_study_dir = 'D:\\Melanoma01\\RT_TraningCohort_convert_PT\\Final_Mel01'
output_root = 'D:\\Melanoma01\\RT_TraningCohort_convert_PT\\Final_Mel01_nii'

In [6]:
# # remove space in folder name
# scan_dir = os.listdir(root_study_dir)
# scan_dir.sort()
# i = 0
# for scan in scan_dir:
#     os.rename(os.path.join(root_study_dir,scan),os.path.join(root_study_dir,scan.replace(" ", "")))

In [8]:
scan_dir = os.listdir(root_study_dir)
scan_dir.sort()
# i = 0
for scan in scan_dir:
    print(scan)
    study_dir = os.listdir(os.path.join(root_study_dir,scan))
    study_dir.sort()
    for study in study_dir:
        scan_modal = os.listdir(os.path.join(root_study_dir,scan,study))
        scan_modal.sort()
        for scan_modal_each in scan_modal:
            
            # image_folder_type = os.listdir(os.path.join(root_study_dir,scan,study,scan_modal_each))

            # for folder_type in image_folder_type:
            # if 'CT' in scan_modal_each:
            #     # converts directory of CT/PET dicom to nii
            #     full_path = os.path.join(root_study_dir,scan,study,scan_modal_each)
            #     dicom2nifti.convert_directory(full_path, output_folder)
            #     print('written CT: ', output_folder)
            # elif 'PT' in scan_modal_each:
            #     # converts directory of CT/PET dicom to nii
            #     full_path = os.path.join(root_study_dir,scan,study,scan_modal_each)
            #     dicom2nifti.convert_directory(full_path, output_folder)
            #     print('written PT: ', output_folder)
            if 'RTst' in scan_modal_each:
                output_folder = os.path.join(output_root,scan_modal_each)
                os.mkdir(output_folder)
                full_path = os.path.join(root_study_dir,scan,study,scan_modal_each)
                rt_file = sorted(glob.glob(full_path + '\\*.dcm'))
                dicom_file_path = sorted(glob.glob(os.path.join(root_study_dir,scan,study) + '\\*_PT_*'))
                # converts rt struct dicom file to nii but needs dicom folder
                dcmrtstruct2nii(rt_file[0], dicom_file_path[0], output_folder)
                print('written RT: ', output_folder)

MEL00587_17012019
written RT:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nii\MEL00587_MEL00587_RTst_2019-01-17_090330_PET.WB.Melanoma_Final_n1__00000
MEL00587_29042019
written RT:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nii\MEL00587_MEL00587_RTst_2019-04-29_094649_PET.WB.Melanoma_Final_n1__00000
MEL00610_14082017
written RT:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nii\MEL00610_MEL00610_RTst_2017-08-14_132824_PET.WB.Melanoma_Final_n1__00000
MEL00610_27112017
written RT:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nii\MEL00610_MEL00610_RTst_2017-11-27_133943_PET.WB.Melanoma_Final_n1__00000
MEL00616_01042016
written RT:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nii\MEL00616_MEL00616_RTst_2016-04-01_122321_PET.WB.Melanoma_Final_n1__00000
MEL00616_16122015
written RT:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nii\MEL00616_MEL00616_RTst_2015-12-16_141806_PET.WB.Melanoma_Final_n1__00000
MEL00627_22012019
written RT

In [60]:
def generate_nrrd_header(ds,img3d):
    # type, dimensions, space = 'left-posterior-superior', sizes, space direction, encoding, endian, kinds, space origin
    z_max_pos = ds[0].ImagePositionPatient[2]
    list_z_pos = []
    for ds_ct in ds:
        list_z_pos.append(ds_ct.ImagePositionPatient[2])
        if ds_ct.ImagePositionPatient[2] < z_max_pos:
            z_max_pos = ds_ct.ImagePositionPatient[2]

    list_z_pos.sort()
    diff = [list_z_pos[i+1]-list_z_pos[i] for i in range(len(list_z_pos)-1)]
    thickness = max(set(diff), key=diff.count)

    image_start = np.asarray((ds[0].ImagePositionPatient[0], ds[0].ImagePositionPatient[1], z_max_pos))
    nrrd_header = OrderedDict()

    nrrd_header['type'] = 'short'  # this gets overridden by the img3d data type
    nrrd_header['dimension'] = 3
    nrrd_header['space'] = 'left-posterior-superior'

    nrrd_header['sizes'] = np.array(img3d.shape)
    nrrd_header['space directions'] = np.array([[ds[0].PixelSpacing[0], 0, 0],
                                                [0, ds[0].PixelSpacing[1], 0],
                                                [0, 0, thickness]])
    nrrd_header['encoding'] = 'gzip'
    nrrd_header['endian'] = 'little'
    nrrd_header['kinds'] = ('domain', 'domain', 'domain')
    nrrd_header['space origin'] = np.array(image_start)

    return nrrd_header

In [61]:
root_study_dir = 'D:\\Melanoma01\\RT_TraningCohort_convert_PT\\Final_Mel01_nii'
output_root = 'D:\\Melanoma01\\RT_TraningCohort_convert_PT\\Final_Mel01_nrrd'

In [58]:
# validate slice thickness
subjects_scans = [os.path.basename(f) for f in sorted(glob.glob(root_study_dir + '\\MEL*'))]
# temp_subjects = [os.path.basename(f) for f in sorted(glob.glob('temp_processing/' + 'PS*/PS*'))]
for sub in subjects_scans:
    print('subject: ', sub)
    nii_files = sorted(glob.glob(root_study_dir + "\\"+ sub + '\\*.nii.*'))
    patient_id = sub.split('_')[0]
    date = sub.split('_')[3]
    date_path = date.split("-")[2] + date.split("-")[1] + date.split("-")[0]
    dicom_dir = sorted(glob.glob(os.path.join("\\".join(root_study_dir.split('\\')[0:-1]), "Final_Mel01",patient_id + "_" + date_path) + "\\*__Studies" + '\\*_PT_*'))
    ref_dicom = [pydicom.dcmread(f) for f in sorted(glob.glob(dicom_dir[0] + '\\2*.dcm'))]

    z_max_pos = ref_dicom[0].ImagePositionPatient[2]
    list_z_pos = []
    for ds_ct in ref_dicom:
        list_z_pos.append(ds_ct.ImagePositionPatient[2])
        if ds_ct.ImagePositionPatient[2] < z_max_pos:
            z_max_pos = ds_ct.ImagePositionPatient[2]

    list_z_pos.sort()
    diff = [list_z_pos[i+1]-list_z_pos[i] for i in range(len(list_z_pos)-1)]
    thickness = max(set(diff), key=diff.count)
    print('Calculated Thickness: ', thickness)
    print('Header Thickness: ', ref_dicom[0].SliceThickness)



subject:  MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Melanoma_Final_n1__00000
Calculated Thickness:  2.0
Header Thickness:  5
subject:  MEL00056_MEL00056_RTst_2016-09-06_121022_PET.WB.Melanoma_Final_n1__00000
Calculated Thickness:  2.0
Header Thickness:  5
subject:  MEL00154_MEL00154_RTst_2015-06-18_123349_PET.WB.Melanoma_Final_n1__00000
Calculated Thickness:  5.0
Header Thickness:  5
subject:  MEL00154_MEL00154_RTst_2015-10-05_141526_PET.WB.Melanoma_Final_n1__00000
Calculated Thickness:  2.0
Header Thickness:  5
subject:  MEL00181_MEL00181_RTst_2018-08-03_130236_PET.WB.Melanoma_Final_n1__00000
Calculated Thickness:  2.0
Header Thickness:  5
subject:  MEL00181_MEL00181_RTst_2018-11-26_083911_PET.WB.Melanoma_Final_n1__00000
Calculated Thickness:  5.0
Header Thickness:  5
subject:  MEL00240_MEL00240_RTst_2014-11-13_123524_PET.WB.Melanoma_Final_n1__00000
Calculated Thickness:  5.0
Header Thickness:  5
subject:  MEL00240_MEL00240_RTst_2015-02-25_103052_PET.WB.Melanoma_Final_n1__00000


In [63]:
# Convert nii to nrrd
subjects_scans = [os.path.basename(f) for f in sorted(glob.glob(root_study_dir + '\\MEL*'))]
# temp_subjects = [os.path.basename(f) for f in sorted(glob.glob('temp_processing/' + 'PS*/PS*'))]
for sub in subjects_scans:
    print('subject: ', sub)
    nii_files = sorted(glob.glob(root_study_dir + "\\"+ sub + '\\*.nii.*'))
    patient_id = sub.split('_')[0]
    date = sub.split('_')[3]
    date_path = date.split("-")[2] + date.split("-")[1] + date.split("-")[0]
    dicom_dir = sorted(glob.glob(os.path.join("\\".join(root_study_dir.split('\\')[0:-1]), "Final_Mel01",patient_id + "_" + date_path) + "\\*__Studies" + '\\*_PT_*'))
    #### dicom_dir = sorted(glob.glob(os.path.join("\\".join(root_study_dir.split('\\')[0:-1]), "Final_Mel01",patient_id + "_" + date_path, "-".join(date.split("-")[0:2]) + "__Studies") + '\\*_PT_*'))
    ref_dicom = [pydicom.dcmread(f) for f in sorted(glob.glob(dicom_dir[0] + '\\2*.dcm'))]
    output_folder = os.path.join(output_root, sub)
    os.mkdir(output_folder)
    print('DICOM: ', dicom_dir)
    
    for nii_file in nii_files:
        a = nib.load(nii_file)
        image3d = a.get_fdata()
        header_info = generate_nrrd_header(ref_dicom, image3d)
        output_file_name = os.path.join(output_folder, os.path.basename(nii_file).split('.')[0] + '.nrrd')
        nrrd.write(output_file_name, image3d, header_info)
        print('wrote: ', output_file_name)

subject:  MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Melanoma_Final_n1__00000
DICOM:  ['D:\\Melanoma01\\RT_TraningCohort_convert_PT\\Final_Mel01\\MEL00056_13062016\\2016-06__Studies\\MEL00056_MEL00056_PT_2016-06-13_125648_PET.WB.Melanoma_PET.WB.AC_n929__00000']
wrote:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nrrd\MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Melanoma_Final_n1__00000\image.nrrd
wrote:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nrrd\MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Melanoma_Final_n1__00000\mask_LN34.nrrd
wrote:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nrrd\MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Melanoma_Final_n1__00000\mask_OR07.nrrd
wrote:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nrrd\MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Melanoma_Final_n1__00000\mask_SK50.nrrd
wrote:  D:\Melanoma01\RT_TraningCohort_convert_PT\Final_Mel01_nrrd\MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Me

In [64]:
#normalise nrrd
subjects_scans = [os.path.basename(f) for f in sorted(glob.glob(output_root + '\\MEL*'))]
for scan in subjects_scans:
    list_of_all_mask = sorted(glob.glob(os.path.join(output_root,scan)+'\\*mask*.nrrd'))
    # print(list_of_all_mask)
    for mask in list_of_all_mask:
        if 'Burden'  in mask:
            burden_mask = nrrd.read(mask)
            img3_normalised = (burden_mask[0] - np.min(burden_mask[0]))/(np.max(burden_mask[0]) - np.min(burden_mask[0]))
            header = burden_mask[1]
            filename = os.path.join(output_root,scan, 'lesionData.nrrd') 
            nrrd.write(filename, img3_normalised, header)  # str, data, header
            print('written combined lesion mask: ', scan)

written combined lesion mask:  MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Melanoma_Final_n1__00000
written combined lesion mask:  MEL00056_MEL00056_RTst_2016-09-06_121022_PET.WB.Melanoma_Final_n1__00000
written combined lesion mask:  MEL00154_MEL00154_RTst_2015-06-18_123349_PET.WB.Melanoma_Final_n1__00000
written combined lesion mask:  MEL00154_MEL00154_RTst_2015-10-05_141526_PET.WB.Melanoma_Final_n1__00000
written combined lesion mask:  MEL00181_MEL00181_RTst_2018-08-03_130236_PET.WB.Melanoma_Final_n1__00000
written combined lesion mask:  MEL00181_MEL00181_RTst_2018-11-26_083911_PET.WB.Melanoma_Final_n1__00000
written combined lesion mask:  MEL00240_MEL00240_RTst_2014-11-13_123524_PET.WB.Melanoma_Final_n1__00000
written combined lesion mask:  MEL00240_MEL00240_RTst_2015-02-25_103052_PET.WB.Melanoma_Final_n1__00000
written combined lesion mask:  MEL00243_MEL00243_RTst_2014-08-18_144412_PET.WB.Melanoma_Final_n1__00000
written combined lesion mask:  MEL00243_MEL00243_RTst_2014-11-06

In [66]:
source_folder = "D:\\Melanoma01\\RT_TraningCohort_convert_PT\\Final_Mel01_nrrd"
destination = "D:\\Melanoma01\\Training Cohort 1 (70)\\Restructured\\Melanoma01\\PatientData_Processed"

In [75]:
#remove nrrd (name error)
for patient in os.listdir(destination):
    for scans in os.listdir(os.path.join(destination,patient)):
        for files in os.listdir(os.path.join(destination,patient,scans)):
            if "lesion" in files:
                os.remove(os.path.join(destination, patient,scans,files))

In [76]:
dirs = os.listdir(source_folder)
dirs.sort()
i = 1
for sessions in dirs:
    if "_RTst_" in sessions:
        print('Processing: ', sessions)
        print(i)
        for file in os.listdir(os.path.join(source_folder,sessions)):
            if "lesionData" in file:
                patient_id = sessions.split("_")[0]
                if i % 2 == 1:
                    scan_folder = patient_id + "_scan01"
                else:
                    scan_folder = patient_id + "_scan02"
                shutil.copyfile(os.path.join(source_folder,sessions,file), os.path.join(destination,patient_id,scan_folder,scan_folder + "_lesionData.nrrd"))
                print('Moved: ', sessions, 'to', scan_folder)
        i += 1


Processing:  MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Melanoma_Final_n1__00000
1
Moved:  MEL00056_MEL00056_RTst_2016-06-13_125648_PET.WB.Melanoma_Final_n1__00000 to MEL00056_scan01
Processing:  MEL00056_MEL00056_RTst_2016-09-06_121022_PET.WB.Melanoma_Final_n1__00000
2
Moved:  MEL00056_MEL00056_RTst_2016-09-06_121022_PET.WB.Melanoma_Final_n1__00000 to MEL00056_scan02
Processing:  MEL00154_MEL00154_RTst_2015-06-18_123349_PET.WB.Melanoma_Final_n1__00000
3
Moved:  MEL00154_MEL00154_RTst_2015-06-18_123349_PET.WB.Melanoma_Final_n1__00000 to MEL00154_scan01
Processing:  MEL00154_MEL00154_RTst_2015-10-05_141526_PET.WB.Melanoma_Final_n1__00000
4
Moved:  MEL00154_MEL00154_RTst_2015-10-05_141526_PET.WB.Melanoma_Final_n1__00000 to MEL00154_scan02
Processing:  MEL00181_MEL00181_RTst_2018-08-03_130236_PET.WB.Melanoma_Final_n1__00000
5
Moved:  MEL00181_MEL00181_RTst_2018-08-03_130236_PET.WB.Melanoma_Final_n1__00000 to MEL00181_scan01
Processing:  MEL00181_MEL00181_RTst_2018-11-26_083911_PET.WB